In [1]:
import json
import os
import time
import random
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from torchvision import models
from torchsummary import summary
from sklearn.model_selection import train_test_split

from model_pytorch import EfficientNet
from utils import Bar,Logger, AverageMeter, accuracy, mkdir_p, savefig
from warmup_scheduler import GradualWarmupScheduler

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import collections

In [2]:
# GPU Device
gpu_id = 0
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_id)
use_cuda = torch.cuda.is_available()
print("GPU device %d:" %(gpu_id), use_cuda)

GPU device 0: True


# Arguments

In [3]:
data_dir = '/media/data2/dataset/GAN_ImageData/StarGAN_128/'

In [4]:
pretrained = ''
resume = ''

In [5]:
# Model
model_name = 'efficientnet-b0' # b0-b7 scale

# Optimization
num_classes = 2
epochs = 200
start_epoch = 0
train_batch = 200
test_batch = 200
lr = 0.015
schedule = [20, 75, 125, 175]
momentum = 0.9
gamma = 0.1 # LR is multiplied by gamma on schedule

# CheckPoint
checkpoint = './log/star/128/b0' # dir
if not os.path.isdir(checkpoint):
    os.makedirs(checkpoint)
num_workers = 4

# Seed
manual_seed = 7
random.seed(manual_seed)
torch.cuda.manual_seed_all(manual_seed)

# Image
size = (128, 128)

best_acc = 0

In [6]:
state = {}
state['num_classes'] = num_classes
state['epochs'] = epochs
state['start_epoch'] = start_epoch
state['train_batch'] = train_batch
state['test_batch'] = test_batch
state['lr'] = lr
state['schedule'] = schedule
state['momentum'] = momentum
state['gamma'] = gamma

# Dataset

In [7]:
train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'validation')    
train_aug = transforms.Compose([
#     transforms.RandomAffine(degrees=2, translate=(0.02, 0.02), scale=(0.98, 1.02), shear=2, fillcolor=(124,117,104)),
    transforms.Resize(size),
#     transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
#     transforms.RandomErasing(p=0.3, scale=(0.02, 0.10), ratio=(0.3, 3.3), value=0, inplace=True),
])
val_aug = transforms.Compose([
    transforms.Resize(size),
    transforms.ToTensor(),
])

# pin_memory : cuda pin memeory use
train_loader = DataLoader(datasets.ImageFolder(train_dir, transform=train_aug),
                          batch_size=train_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_loader = DataLoader(datasets.ImageFolder(val_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)

# Model

In [8]:
model = EfficientNet.from_name(model_name, num_classes=num_classes,
                              override_params={'dropout_rate':0.2, 'drop_connect_rate':0.2})

# Pre-trained
if pretrained:
    print("=> using pre-trained model '{}'".format(pretrained))
    model.load_state_dict(torch.load(pretrained)['state_dict'])

In [9]:
model.to('cuda')
cudnn.benchmark = True
print('    Total params: %.2fM' % (sum(p.numel() for p in model.parameters())/1000000.0))

    Total params: 4.01M


In [10]:
# summary(model, input_size=(3,64,64), device='cuda')

# Loss

In [11]:
criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=1e-4)
# optimizer = optim.Adam(model.parameters(), weight_decay=1e-4)
scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
scheduler_warmup = GradualWarmupScheduler(optimizer, multiplier=8, total_epoch=10, after_scheduler=scheduler_cosine)

In [12]:
# Resume
if resume:
    print('==> Resuming from checkpoint..')
    checkpoint = os.path.dirname(resume)
#     checkpoint = torch.load(resume)
    resume = torch.load(resume)
    best_acc = resume['best_acc']
    start_epoch = resume['epoch']
    model.load_state_dict(resume['state_dict'])
    optimizer.load_state_dict(resume['optimizer'])
    logger = Logger(os.path.join(checkpoint, 'log.txt'), resume=True)
else:
    logger = Logger(os.path.join(checkpoint, 'log.txt'))
    logger.set_names(['Learning Rate', 'Train Loss', 'Valid Loss', 'Train Acc.', 'Valid Acc.'])

# Train

In [13]:
def train(train_loader, model, criterion, optimizer, epoch, use_cuda):
    model.train()
    torch.set_grad_enabled(True)
    
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    end = time.time()
    
    bar = Bar('Processing', max=len(train_loader))
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        batch_size = inputs.size(0)
        if batch_size < train_batch:
            continue
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)

        # compute output
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        bar.suffix  = '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | ETA: {eta:} | Loss: {loss:.4f} | top1: {top1: .4f} '.format(
                    batch=batch_idx + 1,
                    size=len(train_loader),
                    data=data_time.val,
                    bt=batch_time.val,
                    total=bar.elapsed_td,
                    eta=bar.eta_td,
                    loss=losses.avg,
                    top1=top1.avg,
                    )
        bar.next()
        if batch_idx % 10 == 0:
            print('{batch}/{size} Data:{data:.3f} | Batch:{bt:.3f} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:}'.format(
                 batch=batch_idx+1, size=len(train_loader), data=data_time.val, bt=batch_time.val, total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg))
    bar.finish()
    return (losses.avg, top1.avg)

In [14]:
def test(val_loader, model, criterion, epoch, use_cuda):
    global best_acc

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()
    torch.set_grad_enabled(False)

    end = time.time()
    bar = Bar('Processing', max=len(val_loader))
    for batch_idx, (inputs, targets) in enumerate(val_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs, volatile=True), torch.autograd.Variable(targets)

        # compute output
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        bar.suffix  = '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | ETA: {eta:} | Loss: {loss:} | top1: {top1:}'.format(
                    batch=batch_idx + 1,
                    size=len(val_loader),
                    data=data_time.avg,
                    bt=batch_time.avg,
                    total=bar.elapsed_td,
                    eta=bar.eta_td,
                    loss=losses.avg,
                    top1=top1.avg,)
        bar.next()
    print('{batch}/{size} Data:{data:.3f} | Batch:{bt:.3f} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:}'.format(
         batch=batch_idx+1, size=len(val_loader), data=data_time.val, bt=batch_time.val, total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg))
    bar.finish()
    return (losses.avg, top1.avg)

In [15]:
def save_checkpoint(state, is_best, checkpoint='checkpoint', filename='checkpoint.pth.tar'):
    filepath = os.path.join(checkpoint, filename)
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(checkpoint, 'model_best.pth.tar'))

def adjust_learning_rate(optimizer, epoch):
    global state
    lr_set = [1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5]
    lr_list = schedule.copy()
    lr_list.append(epoch)
    lr_list.sort()
    idx = lr_list.index(epoch)
    state['lr'] *= lr_set[idx]
    for param_group in optimizer.param_groups:
        param_group['lr'] = state['lr']

In [ ]:
for epoch in range(start_epoch, epochs):
    state['lr'] = optimizer.state_dict()['param_groups'][0]['lr']
    adjust_learning_rate(optimizer, epoch)
    print('\nEpoch: [%d | %d] LR: %f' % (epoch + 1, epochs, state['lr']))
    train_loss, train_acc = train(train_loader, model, criterion, optimizer, epoch, use_cuda)
    test_loss, test_acc = test(val_loader, model, criterion, epoch, use_cuda)
    
    logger.append([state['lr'], train_loss, test_loss, train_acc, test_acc])
    scheduler_warmup.step()

    is_best = test_acc > best_acc
    best_acc = max(test_acc, best_acc)
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict' : model.state_dict(),
        'acc': test_acc,
        'best_acc': best_acc,
        'optimizer': optimizer.state_dict(),
    }, is_best, checkpoint=checkpoint)


Epoch: [1 | 200] LR: 0.015000
1/1373 Data:1.671 | Batch:6.330 | Total:0:00:06 | ETA:2:24:45 | Loss:0.7153679728507996 | top1:46.0
11/1373 Data:0.063 | Batch:0.611 | Total:0:00:12 | ETA:0:26:37 | Loss:0.7059031995860013 | top1:49.40909194946289
21/1373 Data:0.044 | Batch:0.626 | Total:0:00:18 | ETA:0:12:49 | Loss:0.7024758656819662 | top1:50.35714340209961
31/1373 Data:0.042 | Batch:0.525 | Total:0:00:23 | ETA:0:12:40 | Loss:0.7020594977563427 | top1:50.709678649902344
41/1373 Data:0.041 | Batch:0.553 | Total:0:00:28 | ETA:0:10:49 | Loss:0.7036498566953148 | top1:50.35365676879883
51/1373 Data:0.053 | Batch:0.458 | Total:0:00:33 | ETA:0:11:48 | Loss:0.7028509600489747 | top1:50.13725280761719
61/1373 Data:0.022 | Batch:0.479 | Total:0:00:39 | ETA:0:11:29 | Loss:0.7065665223559395 | top1:49.819671630859375
71/1373 Data:0.043 | Batch:0.478 | Total:0:00:44 | ETA:0:11:49 | Loss:0.7057354257140361 | top1:49.767608642578125
81/1373 Data:0.030 | Batch:0.512 | Total:0:00:50 | ETA:0:13:21 | Los

721/1373 Data:0.049 | Batch:0.623 | Total:0:06:25 | ETA:0:06:13 | Loss:0.7007992997579535 | top1:50.072818756103516
731/1373 Data:0.001 | Batch:0.385 | Total:0:06:30 | ETA:0:06:15 | Loss:0.70072640178957 | top1:50.08002853393555
741/1373 Data:0.031 | Batch:0.498 | Total:0:06:36 | ETA:0:06:00 | Loss:0.7007282789741289 | top1:50.077598571777344
751/1373 Data:0.048 | Batch:0.532 | Total:0:06:42 | ETA:0:06:03 | Loss:0.7007264304415046 | top1:50.08788299560547
761/1373 Data:0.035 | Batch:0.656 | Total:0:06:47 | ETA:0:05:25 | Loss:0.7008494039246037 | top1:50.06175994873047
771/1373 Data:0.026 | Batch:0.531 | Total:0:06:53 | ETA:0:05:26 | Loss:0.7009517264427997 | top1:50.05317687988281
781/1373 Data:0.039 | Batch:0.486 | Total:0:06:58 | ETA:0:05:22 | Loss:0.7008904645293379 | top1:50.07490539550781
791/1373 Data:0.042 | Batch:0.523 | Total:0:07:03 | ETA:0:05:02 | Loss:0.7008910461713933 | top1:50.056888580322266
801/1373 Data:0.042 | Batch:0.525 | Total:0:07:09 | ETA:0:05:20 | Loss:0.700868

/home/cutz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


153/153 Data:0.002 | Batch:0.760 | Total:0:00:26 | ETA:0:00:00 | Loss:0.6931460927073327 | top1:49.99017333984375

Epoch: [2 | 200] LR: 0.025500
1/1373 Data:0.835 | Batch:1.353 | Total:0:00:01 | ETA:0:31:52 | Loss:0.6915422081947327 | top1:54.0
11/1373 Data:0.041 | Batch:0.495 | Total:0:00:06 | ETA:0:13:31 | Loss:0.6945689645680514 | top1:50.272727966308594
21/1373 Data:0.041 | Batch:0.595 | Total:0:00:11 | ETA:0:11:53 | Loss:0.6942876463844663 | top1:50.66666793823242
31/1373 Data:0.052 | Batch:0.528 | Total:0:00:16 | ETA:0:11:31 | Loss:0.694914037181485 | top1:50.24193572998047
41/1373 Data:0.046 | Batch:0.511 | Total:0:00:21 | ETA:0:11:03 | Loss:0.6970871468869652 | top1:49.719512939453125
51/1373 Data:0.041 | Batch:0.481 | Total:0:00:26 | ETA:0:11:33 | Loss:0.6964512362199671 | top1:50.088233947753906
61/1373 Data:0.029 | Batch:0.504 | Total:0:00:32 | ETA:0:11:01 | Loss:0.697175003466059 | top1:49.78688430786133
71/1373 Data:0.027 | Batch:0.600 | Total:0:00:37 | ETA:0:11:09 | Loss:

711/1373 Data:0.035 | Batch:0.509 | Total:0:06:24 | ETA:0:05:58 | Loss:0.6952219569398046 | top1:49.94514846801758
721/1373 Data:0.034 | Batch:0.542 | Total:0:06:30 | ETA:0:05:39 | Loss:0.6952086346490043 | top1:49.934814453125
731/1373 Data:0.062 | Batch:0.615 | Total:0:06:35 | ETA:0:05:49 | Loss:0.6951524383718436 | top1:49.988372802734375
741/1373 Data:0.033 | Batch:0.623 | Total:0:06:41 | ETA:0:05:50 | Loss:0.6951571345007532 | top1:49.993927001953125
751/1373 Data:0.041 | Batch:0.448 | Total:0:06:46 | ETA:0:05:12 | Loss:0.6951546108675066 | top1:50.0159797668457
761/1373 Data:0.042 | Batch:0.589 | Total:0:06:51 | ETA:0:05:47 | Loss:0.6951613976043721 | top1:50.036136627197266
771/1373 Data:0.050 | Batch:0.517 | Total:0:06:57 | ETA:0:05:49 | Loss:0.6951425149746906 | top1:50.025291442871094
781/1373 Data:0.036 | Batch:0.482 | Total:0:07:02 | ETA:0:05:13 | Loss:0.6951425487650189 | top1:50.01152420043945
791/1373 Data:0.030 | Batch:0.471 | Total:0:07:08 | ETA:0:05:13 | Loss:0.695133

...

561/1373 Data:0.061 | Batch:0.597 | Total:0:04:49 | ETA:0:07:30 | Loss:0.0008632563459576953 | top1:99.9732666015625
571/1373 Data:0.032 | Batch:0.471 | Total:0:04:53 | ETA:0:06:15 | Loss:0.0008666943917879728 | top1:99.97285461425781
581/1373 Data:0.037 | Batch:0.493 | Total:0:04:58 | ETA:0:06:27 | Loss:0.0008561164611760064 | top1:99.97332000732422
591/1373 Data:0.042 | Batch:0.431 | Total:0:05:03 | ETA:0:06:33 | Loss:0.000843939464437106 | top1:99.9737777709961
601/1373 Data:0.042 | Batch:0.544 | Total:0:05:08 | ETA:0:06:27 | Loss:0.0008336598350581332 | top1:99.97420501708984
611/1373 Data:0.043 | Batch:0.435 | Total:0:05:13 | ETA:0:06:44 | Loss:0.0008917155359913508 | top1:99.97380828857422
621/1373 Data:0.037 | Batch:0.624 | Total:0:05:19 | ETA:0:06:56 | Loss:0.0008836502109410731 | top1:99.97423553466797
631/1373 Data:0.041 | Batch:0.555 | Total:0:05:24 | ETA:0:06:35 | Loss:0.0008721310650496157 | top1:99.97463989257812
641/1373 Data:0.047 | Batch:0.543 | Total:0:05:29 | ETA:0:0

1261/1373 Data:0.049 | Batch:0.521 | Total:0:10:40 | ETA:0:00:57 | Loss:0.000989370031024641 | top1:99.9698715209961
1271/1373 Data:0.044 | Batch:0.483 | Total:0:10:45 | ETA:0:00:51 | Loss:0.0009844576498505642 | top1:99.97010040283203
1281/1373 Data:0.044 | Batch:0.534 | Total:0:10:50 | ETA:0:00:46 | Loss:0.0010071238009020777 | top1:99.96916198730469
1291/1373 Data:0.041 | Batch:0.502 | Total:0:10:55 | ETA:0:00:42 | Loss:0.0010067073314025598 | top1:99.9690170288086
1301/1373 Data:0.041 | Batch:0.483 | Total:0:11:00 | ETA:0:00:36 | Loss:0.000999738115183478 | top1:99.96925354003906
1311/1373 Data:0.041 | Batch:0.491 | Total:0:11:05 | ETA:0:00:31 | Loss:0.000993604016676414 | top1:99.96949005126953
1321/1373 Data:0.041 | Batch:0.501 | Total:0:11:10 | ETA:0:00:26 | Loss:0.0009968502263862603 | top1:99.96934509277344
1331/1373 Data:0.053 | Batch:0.508 | Total:0:11:15 | ETA:0:00:21 | Loss:0.000991754421529228 | top1:99.96957397460938
1341/1373 Data:0.043 | Batch:0.532 | Total:0:11:20 | E

571/1373 Data:0.043 | Batch:0.498 | Total:0:04:40 | ETA:0:06:36 | Loss:0.0009348627592003928 | top1:99.97636413574219
581/1373 Data:0.041 | Batch:0.486 | Total:0:04:45 | ETA:0:06:32 | Loss:0.0009547248559846846 | top1:99.97589874267578
591/1373 Data:0.041 | Batch:0.492 | Total:0:04:50 | ETA:0:06:27 | Loss:0.0009411642233144895 | top1:99.97631072998047
601/1373 Data:0.038 | Batch:0.496 | Total:0:04:55 | ETA:0:06:23 | Loss:0.0009374563843219602 | top1:99.97669982910156
611/1373 Data:0.041 | Batch:0.500 | Total:0:04:59 | ETA:0:06:17 | Loss:0.0009270337402864219 | top1:99.97708129882812
621/1373 Data:0.041 | Batch:0.497 | Total:0:05:04 | ETA:0:06:15 | Loss:0.0009229755368310641 | top1:99.97665405273438
631/1373 Data:0.041 | Batch:0.485 | Total:0:05:09 | ETA:0:06:08 | Loss:0.0009433926209317935 | top1:99.97543334960938
641/1373 Data:0.041 | Batch:0.492 | Total:0:05:14 | ETA:0:06:01 | Loss:0.0009944863492525717 | top1:99.97504425048828
651/1373 Data:0.039 | Batch:0.501 | Total:0:05:19 | ETA:

1271/1373 Data:0.041 | Batch:0.487 | Total:0:10:26 | ETA:0:00:51 | Loss:0.0009134407227703398 | top1:99.97482299804688
1281/1373 Data:0.041 | Batch:0.492 | Total:0:10:31 | ETA:0:00:46 | Loss:0.0009180425442833718 | top1:99.9746322631836
1291/1373 Data:0.041 | Batch:0.496 | Total:0:10:36 | ETA:0:00:41 | Loss:0.0009178280470336117 | top1:99.97444152832031
1301/1373 Data:0.041 | Batch:0.489 | Total:0:10:41 | ETA:0:00:36 | Loss:0.000914731863243921 | top1:99.97425079345703
1311/1373 Data:0.041 | Batch:0.491 | Total:0:10:46 | ETA:0:00:31 | Loss:0.0009166305544005946 | top1:99.97406768798828
1321/1373 Data:0.036 | Batch:0.490 | Total:0:10:51 | ETA:0:00:26 | Loss:0.0009109388352047777 | top1:99.97425842285156
1331/1373 Data:0.041 | Batch:0.491 | Total:0:10:56 | ETA:0:00:21 | Loss:0.000905539193108691 | top1:99.97445678710938
1341/1373 Data:0.041 | Batch:0.498 | Total:0:11:01 | ETA:0:00:16 | Loss:0.00090461778403833 | top1:99.97427368164062
1351/1373 Data:0.031 | Batch:0.494 | Total:0:11:06 | 

581/1373 Data:0.041 | Batch:0.490 | Total:0:04:44 | ETA:0:06:32 | Loss:0.0008318638545291937 | top1:99.97503662109375
591/1373 Data:0.041 | Batch:0.496 | Total:0:04:49 | ETA:0:06:27 | Loss:0.0008199178890473406 | top1:99.9754638671875
601/1373 Data:0.036 | Batch:0.499 | Total:0:04:54 | ETA:0:06:22 | Loss:0.0008123560801777029 | top1:99.97586822509766
611/1373 Data:0.041 | Batch:0.500 | Total:0:04:59 | ETA:0:06:17 | Loss:0.0008005875367584797 | top1:99.97626495361328
621/1373 Data:0.041 | Batch:0.498 | Total:0:05:04 | ETA:0:06:13 | Loss:0.0007956272743000219 | top1:99.97665405273438
631/1373 Data:0.041 | Batch:0.499 | Total:0:05:09 | ETA:0:06:08 | Loss:0.0007841248865006261 | top1:99.97701263427734
641/1373 Data:0.041 | Batch:0.498 | Total:0:05:14 | ETA:0:06:03 | Loss:0.000776590478439141 | top1:99.97737884521484
651/1373 Data:0.041 | Batch:0.495 | Total:0:05:19 | ETA:0:05:58 | Loss:0.0007689297282151595 | top1:99.97772979736328
661/1373 Data:0.041 | Batch:0.490 | Total:0:05:24 | ETA:0: